In [1]:
# installs with pip install python-dotenv
from dotenv import load_dotenv

#installs needed to use langchain
# pip install huggingface_hub
# pip install openai
# pip install langchain
# from langchain.llms import OpenAI # deprecated?
from langchain.chat_models import ChatOpenAI
from langchain.llms import HuggingFaceHub
from langchain import PromptTemplate, FewShotPromptTemplate
from langchain.chains import LLMChain
import os

load_dotenv()

True

# Basic OpenAI usage with Langchain

In [2]:
help(ChatOpenAI)

Help on class ChatOpenAI in module langchain.chat_models.openai:

class ChatOpenAI(langchain.chat_models.base.BaseChatModel)
 |  ChatOpenAI(*, cache: Optional[bool] = None, verbose: bool = None, callbacks: Union[List[langchain.callbacks.base.BaseCallbackHandler], langchain.callbacks.base.BaseCallbackManager, NoneType] = None, callback_manager: Optional[langchain.callbacks.base.BaseCallbackManager] = None, tags: Optional[List[str]] = None, client: Any = None, model: str = 'gpt-3.5-turbo', temperature: float = 0.7, model_kwargs: Dict[str, Any] = None, openai_api_key: Optional[str] = None, openai_api_base: Optional[str] = None, openai_organization: Optional[str] = None, openai_proxy: Optional[str] = None, request_timeout: Union[float, Tuple[float, float], NoneType] = None, max_retries: int = 6, streaming: bool = False, n: int = 1, max_tokens: Optional[int] = None) -> None
 |  
 |  Wrapper around OpenAI Chat large language models.
 |  
 |  To use, you should have the ``openai`` python pack

In [3]:
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

In [4]:
# note that if openai_api_key is omitted, OpenAI will attempt to get it direct from environment variables
gpt_model = ChatOpenAI(
    model_name="gpt-3.5-turbo-0613", 
    temperature=0.7, 
    max_tokens=300, 
    openai_api_key=OPENAI_API_KEY
)

In [5]:
# print(gpt_model.predict("Give a summary of Von Clauswitz's book 'On War' in less than 100 words"))

# Basic Hugging Face usage with Langchain

In [6]:
HUGGINGFACEHUB_API_TOKEN = os.getenv('HUGGINGFACEHUB_API_TOKEN')

In [8]:
help(HuggingFaceHub)

Help on class HuggingFaceHub in module langchain.llms.huggingface_hub:

class HuggingFaceHub(langchain.llms.base.LLM)
 |  HuggingFaceHub(*, cache: Optional[bool] = None, verbose: bool = None, callbacks: Union[List[langchain.callbacks.base.BaseCallbackHandler], langchain.callbacks.base.BaseCallbackManager, NoneType] = None, callback_manager: Optional[langchain.callbacks.base.BaseCallbackManager] = None, tags: Optional[List[str]] = None, client: Any = None, repo_id: str = 'gpt2', task: Optional[str] = None, model_kwargs: Optional[dict] = None, huggingfacehub_api_token: Optional[str] = None) -> None
 |  
 |  Wrapper around HuggingFaceHub  models.
 |  
 |  To use, you should have the ``huggingface_hub`` python package installed, and the
 |  environment variable ``HUGGINGFACEHUB_API_TOKEN`` set with your API token, or pass
 |  it as a named parameter to the constructor.
 |  
 |  Only supports `text-generation`, `text2text-generation` and `summarization` for now.
 |  
 |  Example:
 |      ..

In [9]:
hf_model = HuggingFaceHub(
    repo_id='google/flan-t5-base',
    model_kwargs={'temperature':0.1},
    huggingfacehub_api_token=HUGGINGFACEHUB_API_TOKEN
)

C:\Users\justi\Anaconda3\envs\langchain\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [10]:
print(hf_model("what is the capital of Finland?"))

sydney


In [11]:
print(hf_model("what would wikipedia say is the capital of Finland?"))

Helsinki


# Demonstrate a langchain prompt

Prompts can consit of several elements
- Context
- Question / Task
- Examples
- output format

In [12]:
template = """
You are a naval historian with experience in the Napoleonic period.
Return a list of ten relevant ship names for ships which existed in this period for {country}.
"""

In [13]:
prompt = PromptTemplate(
    input_variables=["country"],
    template=template
)

In [14]:
# note that if openai_api_key is omitted, OpenAI will attempt to get it direct from environment variables
gpt_model_2 = ChatOpenAI(
    model_name="gpt-3.5-turbo-0613", 
    temperature=0.7, 
    max_tokens=300, 
    openai_api_key=OPENAI_API_KEY
)

In [15]:
# we can use the template direct
print(gpt_model_2.predict(text=prompt.format(country="Britain")))

1. HMS Victory - One of the most famous British warships of the Napoleonic era, it served as Admiral Nelson's flagship at the Battle of Trafalgar in 1805.
2. HMS Royal Sovereign - A first-rate ship of the line, it played a crucial role in the Battle of Cape St Vincent in 1797.
3. HMS Bellerophon - This third-rate ship of the line gained fame for being the vessel that Napoleon Bonaparte surrendered on after the Battle of Waterloo in 1815.
4. HMS Agamemnon - A 64-gun third-rate ship, it was commanded by Nelson during his early career and participated in several major battles.
5. HMS Temeraire - Known as the "Fighting Temeraire," this second-rate ship played a pivotal role in the Battle of Trafalgar and was immortalized in a famous painting by J.M.W. Turner.
6. HMS Dreadnought - A first-rate ship of the line, it was launched in 1801 and served during the Napoleonic Wars, including Trafalgar.
7. HMS Superb - A 74-gun third-rate ship, it was involved in numerous engagements against French a

In [16]:
# or alternatively we can sart to chain out work together
chain = LLMChain(
    llm=gpt_model_2,
    prompt=prompt
)

print(chain.run("France"))

1. Le Vengeur - A 74-gun ship of the line
2. Le Bucentaure - A 80-gun ship of the line
3. L'Orient - A 120-gun ship of the line
4. Le Redoutable - A 74-gun ship of the line
5. Le Formidable - A 80-gun ship of the line
6. Le Neptune - A 74-gun ship of the line
7. L'Aigle - A 74-gun ship of the line
8. Le Fougueux - A 74-gun ship of the line
9. Le Téméraire - A 74-gun ship of the line
10. L'Intrépide - A 74-gun ship of the line


# Few Shot

In [17]:
rhyme_examples = [
    {'word': 'sun', 'rhyme': 'bun'},
    {'word': 'float', 'rhyme': 'goat'},
    {'word': 'remember', 'rhyme': 'november'},
]

rhyme_template = """
Word: {word}
Rhyme: {rhyme}
"""

rhyme_prompt = PromptTemplate(
    input_variables=['word', 'rhyme'],
    template=rhyme_template
)

In [18]:
# we can format for inputs as before but we need to use the fewshot prompt to build the rhyme_examples into a prompt
rhyme_prompt.format(word='word', rhyme='bird')

'\nWord: word\nRhyme: bird\n'

In [19]:
few_shot_prompt = FewShotPromptTemplate(
    # These are the examples we want to insert into the prompt.
    examples=rhyme_examples,
    # This is how we want to format the examples when we insert them into the prompt.
    example_prompt=rhyme_prompt,
    # The prefix is some text that goes before the examples in the prompt.
    # Usually, this consists of intructions.
    prefix="Give a rhyming word for each input",
    # The suffix is some text that goes after the examples in the prompt.
    # Usually, this is where the user input will go
    suffix="""Word: {input}\nRhyme:\n
    """,
    # The input variables are the variables that the overall prompt expects.
    input_variables=["input"],
    # The example_separator is the string we will use to join the prefix, examples, and suffix together with.
    example_separator="\n",
)

In [20]:
print(few_shot_prompt.format(input="round"))

Give a rhyming word for each input

Word: sun
Rhyme: bun


Word: float
Rhyme: goat


Word: remember
Rhyme: november

Word: round
Rhyme:

    


In [21]:
hf_model_2 = HuggingFaceHub(
    repo_id='google/flan-t5-base',
    model_kwargs={'temperature':0.1},
    huggingfacehub_api_token=HUGGINGFACEHUB_API_TOKEN
)

In [22]:
# google flan tends to be terrible
print(hf_model_2(few_shot_prompt.format(input="round")))

round


In [26]:
# google flan tends to be terrible
print(hf_model_2(few_shot_prompt.format(input="cough")))

cough


In [23]:
gpt_model_3 = ChatOpenAI(
    model_name="gpt-3.5-turbo-0613", 
    temperature=0.7, 
    max_tokens=300, 
    openai_api_key=OPENAI_API_KEY
)

In [24]:
# chatGPT is rather better
print(gpt_model_3.predict(few_shot_prompt.format(input="round")))

bound


In [25]:
# chatGPT is rather better
print(gpt_model_3.predict(few_shot_prompt.format(input="enough")))

cough


# Tools
these include
- Python REPL
- Search engine APIs (there is a DuckDuckGo API)
- Wolfram API
- SQL
- Requests chain to get a response from the web

# Chains
- join LLMs, prompts and tools
- pass outputs as next inputs
- similar to ML pipelines

## Types
- Generic chains
- Utility chains
- Asynchronous chains

- Transformation chain - applies code like transformation (eg regex)
- Chains can be made of other chains

CAn run python code, SQL, query 

In [29]:
last_rows = [0,1,2,3,4]
#ast_rows[-1, :]

In [30]:
last_rows

[0, 1, 2, 3, 4]

In [33]:
last_rows[-1:]

[4]

In [37]:
import numpy as np
last_rows = np.random.randn(1, 1)

for i in range(1, 101):
    print('last_rows')
    print(last_rows)
    new_rows = last_rows[-1, :] + np.random.randn(5, 1).cumsum(axis=0)
    print('new_rows')
    print(new_rows)
    #status_text.text("%i%% Complete" % i)
    #chart.add_rows(new_rows)
    #progress_bar.progress(i)
    last_rows = new_rows
    #time.sleep(0.05)
    

last_rows
[[0.57877107]]
new_rows
[[ 1.14146815]
 [ 0.44200552]
 [ 0.37335953]
 [-0.69548122]
 [-0.9493953 ]]
last_rows
[[ 1.14146815]
 [ 0.44200552]
 [ 0.37335953]
 [-0.69548122]
 [-0.9493953 ]]
new_rows
[[-1.04282169]
 [-1.03977773]
 [-1.342834  ]
 [-1.43826013]
 [-1.0251471 ]]
last_rows
[[-1.04282169]
 [-1.03977773]
 [-1.342834  ]
 [-1.43826013]
 [-1.0251471 ]]
new_rows
[[-0.31778677]
 [-0.80532854]
 [-1.59685401]
 [-3.19442716]
 [-3.41181018]]
last_rows
[[-0.31778677]
 [-0.80532854]
 [-1.59685401]
 [-3.19442716]
 [-3.41181018]]
new_rows
[[-2.10651861]
 [-0.85957326]
 [ 0.09785644]
 [-1.99628729]
 [-1.50590078]]
last_rows
[[-2.10651861]
 [-0.85957326]
 [ 0.09785644]
 [-1.99628729]
 [-1.50590078]]
new_rows
[[-2.74877551]
 [-3.29001392]
 [-2.2810073 ]
 [-3.78685056]
 [-4.17211442]]
last_rows
[[-2.74877551]
 [-3.29001392]
 [-2.2810073 ]
 [-3.78685056]
 [-4.17211442]]
new_rows
[[-4.58915112]
 [-5.33769398]
 [-5.79238979]
 [-5.84635922]
 [-4.07620791]]
last_rows
[[-4.58915112]
 [-5.33769